In [1]:
#Import

import os
import cv2

import numpy as np

from skimage import color
from skimage.feature import hog

from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

import joblib


In [2]:
# Load Images

def load_images_and_labels(dataset_path, image_size=(128, 128)):
    images = []
    labels = []

    for label_folder in os.listdir(dataset_path):
        label_path = os.path.join(dataset_path, label_folder)

        for file_name in os.listdir(label_path):
            file_path = os.path.join(label_path, file_name)
            image = cv2.imread(file_path)
            image = cv2.resize(image, image_size)
            images.append(image)
            labels.append(label_folder)

    return images, labels

images, labels = load_images_and_labels("gambar")


In [3]:
# Ekstraksi fitur HOG

def extract_hog_features(images):
    features = []

    for image in images:
        gray = color.rgb2gray(image)
        hog_feature, _ = hog(
            gray,
            orientations=9,
            pixels_per_cell=(8, 8),
            cells_per_block=(2, 2),
            block_norm='L2-Hys',
            visualize=True
        )
        features.append(hog_feature)

    return np.array(features)

hog_features = extract_hog_features(images)


In [4]:
# Extraksi Fitur Color Histogram

def extract_color_histogram_features(images, bins=256):
    features = []

    for image in images:
        chans = cv2.split(image) 
        hist_features = []

        for chan in chans:
            hist = cv2.calcHist([chan], [0], None, [bins], [0, 256])
            hist = cv2.normalize(hist, hist).flatten()
            hist_features.extend(hist)

        features.append(hist_features)

    return np.array(features)

color_histogram_features = extract_color_histogram_features(images)


In [5]:
# Ekstraksi fitur Tekstur(LBP)

def extract_lbp_features(images):
    features = []

    for image in images:
        gray = color.rgb2gray(image)
        gray_uint8 = (gray * 255).astype("uint8")
        lbp = local_binary_pattern(gray_uint8, P=8, R=1, method='uniform')

        (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-6)  # normalisasi

        features.append(hist)

    return np.array(features)

lbp_features = extract_lbp_features(images)


In [6]:
# Gabungkan Fitur

X = np.concatenate([hog_features, color_histogram_features, lbp_features], axis=1)

In [7]:
#Encode Label

le = LabelEncoder()
y = le.fit_transform(labels)

In [8]:
# Split data train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [9]:
# Train Model

clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))


Accuracy: 0.8333333333333334
Classification Report:
                 precision    recall  f1-score   support

Celana_Panjang       0.83      1.00      0.91        10
          Gaun       0.73      0.89      0.80         9
         Jaket       1.00      0.64      0.78        11

      accuracy                           0.83        30
     macro avg       0.85      0.84      0.83        30
  weighted avg       0.86      0.83      0.83        30



In [10]:
# Dump Model

# joblib.dump(clf, 'model_svm.pkl')
# joblib.dump(le, 'label_encoder.pkl')
# print("Model dan LabelEncoder berhasil disimpan.")


In [11]:
# Load Model

# clf = joblib.load('model_svm.pkl')
# le = joblib.load('label_encoder.pkl')
# print("Model dan LabelEncoder berhasil di load.")

In [12]:
# Predict

def predict_image(model, label_encoder, image_path, image_size=(128, 128)):
    image = cv2.imread(image_path)
    image = cv2.resize(image, image_size)
    gray = color.rgb2gray(image)

    # HOG
    hog_feature, _ = hog(
        gray,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm='L2-Hys',
        visualize=True
    )

    # Color Histogram
    chans = cv2.split(image)
    color_hist = []
    for chan in chans:
        hist = cv2.calcHist([chan], [0], None, [256], [0, 256])
        hist = cv2.normalize(hist, hist).flatten()
        color_hist.extend(hist)

    # LBP
    gray_uint8 = (gray * 255).astype("uint8")
    lbp = local_binary_pattern(gray_uint8, P=8, R=1, method='uniform')
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, 10), range=(0, 9))
    lbp_hist = lbp_hist.astype("float")
    lbp_hist /= (lbp_hist.sum() + 1e-6)
    
    combined_features = np.concatenate([hog_feature, color_hist, lbp_hist])
    combined_features = combined_features.reshape(1, -1)
    
    prediction = model.predict(combined_features)
    predicted_label = label_encoder.inverse_transform(prediction)

    print(f"Gambar '{image_path}' diklasifikasikan sebagai: {predicted_label[0]}")


predict_image(clf,le,'test/celana_panjang.jpg')

Gambar 'test/celana_panjang.jpg' diklasifikasikan sebagai: Celana_Panjang
